In [3]:
from keras.datasets import mnist
from keras.layers import Input,Dense,Reshape,Flatten,Dropout
from keras.layers import BatchNormalization,Activation,ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D,Conv2D
from keras.models import Sequential,Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

(x_treino,_),(_,_) = mnist.load_data()

In [4]:
x_treino.shape

(60000, 28, 28)

In [5]:
x_treino = x_treino/127.5 - 1.0 #normalizando entre -1 e 1
x_treino = np.expand_dims(x_treino,axis=3) #criando uma dimensao extra para a profundidade = 1

In [6]:
x_treino.shape

(60000, 28, 28, 1)

In [33]:
class DCGAN():
    def __init__(self):
        #input shape
        self.altura = 28
        self.largura = 28
        self.channels = 1
        self.img_shape = (self.altura,self.largura,self.channels)
        self.dimensao_randomica = 100
        
        optimizer = Adam(lr=0.0002,beta_1=0.5)
        
        #Definindo o discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        
        #Definindo o generator
        self.generator = self.build_generator()
        
        #Definindo o modelo completo
        esqueleto_entrada_randomica = Input(shape=(self.dimensao_randomica,))
        esqueleto_imagem_gerada = self.generator(esqueleto_entrada_randomica)
        
        self.discriminator.trainable = False #impedindo o discriminator de treinar
        verificacoes = self.discriminator(esqueleto_imagem_gerada)
        
        self.gan_completa = Model(esqueleto_entrada_randomica, verificacoes)
        self.gan_completa.compile(loss="binary_crossentropy",optimizer=optimizer)
        
        
    def build_generator(self):
        
        modelo = Sequential()
        
        #O objetivo é finalizar com uma dimensao de 28x28x1
        modelo.add(Dense(128*7*7,activation="relu",input_dim=self.dimensao_randomica))
        modelo.add(Reshape((7,7,128)))
        modelo.add(UpSampling2D()) #drobra a dimensao atual 7x7x128 para 28x28x128
        modelo.add(Conv2D(128,kernel_size=3,padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(Activation("relu"))
        modelo.add(UpSampling2D()) #14x14x128 se torna 28x28x128
        modelo.add(Conv2D(64,kernel_size=3,padding="same"))#28x28x64
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(Activation("relu"))
        modelo.add(Conv2D(self.channels,kernel_size=3,padding="same")) #28x28x1
        modelo.add(Activation("tanh"))
        
        ruido = Input(shape=(self.dimensao_randomica,))
        imagem_gerada = modelo(ruido)
        
        return Model(ruido,imagem_gerada) #definindo quem sera q entrada e a saida da rede neural
    
        
    
    def build_discriminator(self):
        
        modelo = Sequential()
        
        modelo.add(Conv2D(32,kernel_size=3,strides=2,input_shape=self.img_shape,padding="same"))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(64,kernel_size=3,strides=2,padding="same"))
        modelo.add(ZeroPadding2D(padding=((0,1),(0,1))))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(128,kernel_size=3,strides=2,padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Conv2D(256,kernel_size=3,strides=1,padding="same"))
        modelo.add(BatchNormalization(momentum=0.8))
        modelo.add(LeakyReLU(alpha=0.2))
        modelo.add(Dropout(0.25))
        modelo.add(Flatten())
        modelo.add(Dense(1,activation="sigmoid"))
        
        
        imagem_entrada = Input(shape=self.img_shape)
        verificacao = modelo(imagem_entrada)
        
        return Model(imagem_entrada,verificacao)
    
        
    
    def train(self,epochs,batch_size=128,intervalo_registro=50,x_treino=None):
        
        x_treino = x_treino #carregando o dataset
        
        #criando os gabaritos imagens reais=1 imagens fake=0
        uns = np.ones((batch_size,1))
        zeros = np.zeros((batch_size,1))
        
        
        for epoch in range(epochs):
            
            
            "Discriminator"
        #selecionando um batch aleatorio de imagens reais
            indices = np.random.randint(0,x_treino.shape[0],batch_size)
            imagens_reais = x_treino[indices]
            
        #selecionando um batch de ruidos para gerar imagens fakes
            amostras_ruidos = np.random.normal(0,1,(batch_size,self.dimensao_randomica))
            imagens_geradas = self.generator.predict(amostras_ruidos)
            
        #treinando o discriminador para aprender a diferença entre imagens reais e fakes
            d_loss_real = self.discriminator.train_on_batch(imagens_reais,uns)
            d_loss_fake = self.discriminator.train_on_batch(imagens_geradas,zeros)
            discriminator_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
            
            
            "Generator"
        #Treinando o generator para aprender a enganar o discriminador
            generator_loss = self.gan_completa.train_on_batch(amostras_ruidos,uns)
            
        #Mostrando o progresso:
            print("%d [Discriminator loss: %f, acuracia %2.f%%][Generator loss: %f]" %(epoch,discriminator_loss[0],100*discriminator_loss[1],generator_loss))
            
        
        #Salvando as imagens geradas
            if epoch % intervalo_registro == 0:
                self.save_imgs(epoch)
                
                
                
    def save_imgs(self,epoch):
        linhas = 5
        colunas = 5
        amostras_ruidos = np.random.normal(0,1,(linhas*colunas,self.dimensao_randomica))
        imagens_geradas = self.generator.predict(amostras_ruidos)
        
        
        #normalizando as imagens entre [0,1] para mostrar com matplotlib gray
        imagens_geradas = 0.5*imagens_geradas+0.5
        
        fig,axs = plt.subplots(linhas,colunas)
        cont = 0
        for i in range(linhas):
            for j in range(colunas):
                axs[i,j].imshow(imagens_geradas[cont, :,:,0],cmap="gray")
                axs[i,j].axis("off")
                cont+=1
        fig.savefig("C:/Users/User/Desktop/GANs/mnist_%d.png" % epoch)
        plt.close()
        
        
            
        
        
        

In [34]:
import time
gan_convolucional = DCGAN()
inicio = time.time()
gan_convolucional.train(epochs=20000,batch_size=32,intervalo_registro=500,x_treino=x_treino)
print("Tempo em minutos:",(time.time()-inicio)/60 )


0 [Discriminator loss: 1.136640, acuracia 38%][Generator loss: 0.682443]
1 [Discriminator loss: 0.614546, acuracia 67%][Generator loss: 0.697408]
2 [Discriminator loss: 0.277852, acuracia 95%][Generator loss: 0.699489]
3 [Discriminator loss: 0.280817, acuracia 89%][Generator loss: 0.695719]
4 [Discriminator loss: 0.226375, acuracia 98%][Generator loss: 0.706369]
5 [Discriminator loss: 0.452933, acuracia 75%][Generator loss: 0.825421]
6 [Discriminator loss: 0.483725, acuracia 73%][Generator loss: 1.170136]
7 [Discriminator loss: 0.453126, acuracia 78%][Generator loss: 1.207821]
8 [Discriminator loss: 0.663235, acuracia 66%][Generator loss: 1.066076]
9 [Discriminator loss: 0.678197, acuracia 64%][Generator loss: 1.283575]
10 [Discriminator loss: 0.705758, acuracia 62%][Generator loss: 1.128432]
11 [Discriminator loss: 0.486539, acuracia 83%][Generator loss: 1.014257]
12 [Discriminator loss: 0.315969, acuracia 88%][Generator loss: 0.912758]
13 [Discriminator loss: 0.152807, acuracia 98%][

70 [Discriminator loss: 0.579953, acuracia 66%][Generator loss: 1.159961]
71 [Discriminator loss: 0.624732, acuracia 69%][Generator loss: 1.285497]
72 [Discriminator loss: 0.646444, acuracia 66%][Generator loss: 1.474743]
73 [Discriminator loss: 0.908488, acuracia 53%][Generator loss: 1.496701]
74 [Discriminator loss: 0.825705, acuracia 59%][Generator loss: 1.183610]
75 [Discriminator loss: 0.721519, acuracia 62%][Generator loss: 0.915129]
76 [Discriminator loss: 0.750319, acuracia 53%][Generator loss: 0.926239]
77 [Discriminator loss: 0.747645, acuracia 55%][Generator loss: 1.277202]
78 [Discriminator loss: 0.969686, acuracia 41%][Generator loss: 1.405723]
79 [Discriminator loss: 0.796703, acuracia 53%][Generator loss: 1.459780]
80 [Discriminator loss: 0.748965, acuracia 55%][Generator loss: 1.396715]
81 [Discriminator loss: 0.765887, acuracia 56%][Generator loss: 0.892722]
82 [Discriminator loss: 0.634851, acuracia 61%][Generator loss: 1.211033]
83 [Discriminator loss: 0.920452, acur

180 [Discriminator loss: 0.638479, acuracia 58%][Generator loss: 0.811062]
181 [Discriminator loss: 0.749557, acuracia 61%][Generator loss: 0.908596]
182 [Discriminator loss: 0.745097, acuracia 53%][Generator loss: 1.059295]
183 [Discriminator loss: 0.692877, acuracia 59%][Generator loss: 1.094671]
184 [Discriminator loss: 0.761113, acuracia 58%][Generator loss: 0.983449]
185 [Discriminator loss: 0.749560, acuracia 59%][Generator loss: 1.046183]
186 [Discriminator loss: 0.710380, acuracia 64%][Generator loss: 1.267612]
187 [Discriminator loss: 0.655478, acuracia 61%][Generator loss: 1.238756]
188 [Discriminator loss: 0.749753, acuracia 62%][Generator loss: 1.277635]
189 [Discriminator loss: 0.919797, acuracia 41%][Generator loss: 1.088778]
190 [Discriminator loss: 0.864020, acuracia 44%][Generator loss: 1.222929]
191 [Discriminator loss: 0.808848, acuracia 48%][Generator loss: 1.064449]
192 [Discriminator loss: 0.755383, acuracia 56%][Generator loss: 1.031923]
193 [Discriminator loss: 

290 [Discriminator loss: 0.908708, acuracia 45%][Generator loss: 1.011146]
291 [Discriminator loss: 0.799419, acuracia 45%][Generator loss: 1.073851]
292 [Discriminator loss: 0.635832, acuracia 66%][Generator loss: 1.113674]
293 [Discriminator loss: 0.635996, acuracia 64%][Generator loss: 0.975243]
294 [Discriminator loss: 0.608766, acuracia 64%][Generator loss: 1.206406]
295 [Discriminator loss: 0.698900, acuracia 53%][Generator loss: 1.054717]
296 [Discriminator loss: 0.798112, acuracia 58%][Generator loss: 1.082660]
297 [Discriminator loss: 0.821786, acuracia 48%][Generator loss: 1.030776]
298 [Discriminator loss: 0.524472, acuracia 77%][Generator loss: 1.184992]
299 [Discriminator loss: 0.852181, acuracia 42%][Generator loss: 1.065561]
300 [Discriminator loss: 0.847558, acuracia 45%][Generator loss: 0.971683]
301 [Discriminator loss: 0.687158, acuracia 55%][Generator loss: 1.066394]
302 [Discriminator loss: 0.589369, acuracia 73%][Generator loss: 1.295149]
303 [Discriminator loss: 

400 [Discriminator loss: 0.565908, acuracia 69%][Generator loss: 1.056104]
401 [Discriminator loss: 0.632574, acuracia 66%][Generator loss: 1.084673]
402 [Discriminator loss: 0.556299, acuracia 77%][Generator loss: 1.153929]
403 [Discriminator loss: 0.682013, acuracia 64%][Generator loss: 1.089789]
404 [Discriminator loss: 0.622903, acuracia 64%][Generator loss: 1.140602]
405 [Discriminator loss: 0.741836, acuracia 55%][Generator loss: 1.187399]
406 [Discriminator loss: 0.740181, acuracia 56%][Generator loss: 1.115352]
407 [Discriminator loss: 0.717233, acuracia 58%][Generator loss: 1.080839]
408 [Discriminator loss: 0.752193, acuracia 53%][Generator loss: 1.148825]
409 [Discriminator loss: 0.538468, acuracia 70%][Generator loss: 1.317203]
410 [Discriminator loss: 0.799520, acuracia 53%][Generator loss: 0.989352]
411 [Discriminator loss: 0.631312, acuracia 67%][Generator loss: 0.837932]
412 [Discriminator loss: 0.783410, acuracia 53%][Generator loss: 1.189562]
413 [Discriminator loss: 

...

19480 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007552]
19481 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.005400]
19482 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.006431]
19483 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.004751]
19484 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.016267]
19485 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.002828]
19486 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.006594]
19487 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.003491]
19488 [Discriminator loss: 0.000004, acuracia 100%][Generator loss: 0.009938]
19489 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.008374]
19490 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007365]
19491 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009656]
19492 [Discriminator loss: 0.000000, acuracia 100%][Generator lo

19586 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.008376]
19587 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014927]
19588 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.006791]
19589 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.016880]
19590 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014503]
19591 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007118]
19592 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.012709]
19593 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.008839]
19594 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009474]
19595 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.020848]
19596 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.016459]
19597 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014046]
19598 [Discriminator loss: 0.000000, acuracia 100%][Generator lo

19692 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.031934]
19693 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.012754]
19694 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.020551]
19695 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.013358]
19696 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.019700]
19697 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009885]
19698 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.015537]
19699 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.019156]
19700 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.017874]
19701 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.026346]
19702 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.015070]
19703 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.019749]
19704 [Discriminator loss: 0.000000, acuracia 100%][Generator lo

19798 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.015107]
19799 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009190]
19800 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009182]
19801 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007122]
19802 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.010020]
19803 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.008996]
19804 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007055]
19805 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014177]
19806 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.010704]
19807 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.011692]
19808 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009909]
19809 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.022691]
19810 [Discriminator loss: 0.000000, acuracia 100%][Generator lo

19904 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009511]
19905 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.010242]
19906 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.005684]
19907 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.013506]
19908 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.007116]
19909 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014325]
19910 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.004041]
19911 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.014208]
19912 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.010010]
19913 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.009949]
19914 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.003274]
19915 [Discriminator loss: 0.000000, acuracia 100%][Generator loss: 0.005597]
19916 [Discriminator loss: 0.000000, acuracia 100%][Generator lo